Hybrid Checker


In [61]:
import pandas as pd

# Sample data
data = {'id': [1, 2, 3], 'value': ['hybrid','non-hybrid','non-hybrid']}
df = pd.DataFrame(data)

# # Modify the logic to mark both rows as 'failed' when there are consecutive 'hybrid' values
# df['status'] = df['value'].eq('hybrid') & (df['value'].shift(1).eq('hybrid') | df['value'].shift(-1).eq('hybrid'))

# # Apply the 'failed' and 'ok' labels
# df['status'] = df['status'].apply(lambda x: 'failed' if x else 'ok')

# Initialize status column as empty
df['status'] = ''

# Loop through and check for less than 3-row difference
for i in range(len(df)):
    if df.loc[i, 'value'] == 'hybrid':
        # Check if there is a 'pano' in the previous 1 or 2 rows or next 1 or 2 rows
        for j in range(1, 2):  # Loop for 1 and 2 row difference
            if (i - j >= 0 and df.loc[i - j, 'value'] == 'hybrid') or (i + j < len(df) and df.loc[i + j, 'value'] == 'hybrid'):
                df.loc[i, 'status'] = 'failed'

# Fill remaining empty statuses with 'ok' for non-pano rows
df['status'] = df['status'].replace('', 'ok')

In [62]:
df

,id,value,status
0,1,hybrid,ok
1,2,non-hybrid,ok
2,3,non-hybrid,ok


PANO checker


In [47]:
# Correct the logic to ensure only 'pano' rows are marked 'failed' and 'non-pano' rows remain 'ok'

# New sample data
data = {'id': [1, 2, 3, 4, 5], 'value': ['pano', 'pano', 'non-pano', 'pano', 'non-pano']}
df = pd.DataFrame(data)

# Initialize status column as empty
df['status'] = ''

# Loop through and check for less than 3-row difference
for i in range(len(df)):
    if df.loc[i, 'value'] == 'pano':
        # Check if there is a 'pano' in the previous 1 or 2 rows or next 1 or 2 rows
        for j in range(1, 4):  # Loop for 1 and 2 row difference
            if (i - j >= 0 and df.loc[i - j, 'value'] == 'pano') or (i + j < len(df) and df.loc[i + j, 'value'] == 'pano'):
                df.loc[i, 'status'] = 'failed'

# Fill remaining empty statuses with 'ok' for non-pano rows
df['status'] = df['status'].replace('', 'ok')



In [48]:
df

,id,value,status
0,1,pano,failed
1,2,pano,failed
2,3,non-pano,ok
3,4,pano,failed
4,5,non-pano,ok


CSA Checker


In [57]:
# Correct the logic to ensure only 'pano' rows are marked 'failed' and 'non-pano' rows remain 'ok'

# New sample data
data = {'id': [1, 2, 3, 4, 5, 6, 7], 'value': ['csa', 'csa', 'non-csa', 'non-csa', 'non-csa','non-csa', 'csa']}
df = pd.DataFrame(data)

# Initialize status column as empty
df['status'] = ''

# Loop through and check for less than 3-row difference
for i in range(len(df)):
    if df.loc[i, 'value'] == 'csa':
        # Check if there is a 'pano' in the previous 1 or 2 rows or next 1 or 2 rows
        for j in range(1, 5):  # Loop for 1 and 2 row difference
            if (i - j >= 0 and df.loc[i - j, 'value'] == 'csa') or (i + j < len(df) and df.loc[i + j, 'value'] == 'csa'):
                df.loc[i, 'status'] = 'failed'

# Fill remaining empty statuses with 'ok' for non-pano rows
df['status'] = df['status'].replace('', 'ok')



In [58]:
df

,id,value,status
0,1,csa,failed
1,2,csa,failed
2,3,non-csa,ok
3,4,non-csa,ok
4,5,non-csa,ok
5,6,non-csa,ok
6,7,csa,ok


In [81]:
import pandas as pd

# New sample data
data = {
    'id': [1, 2, 3, 4, 5, 6, 7], 
    'hybrid': ['hybrid', 'non-hybrid', 'non-hybrid', 'hybrid', 'non-hybrid', 'non-hybrid','hybrid'],
    'pano': ['pano', 'non-pano', 'non-pano', 'non-pano', 'pano', 'pano', 'pano'],
    'csa': ['csa', 'non-csa', 'csa', 'non-csa', 'non-csa','non-csa', 'csa']
}
df = pd.DataFrame(data)

# Initialize status column as empty
df['status'] = ''
df['failure_reason'] = ''

# Function to check and append the failure reason
def check_failure(df, column_name, row, j_range, reason):
    for j in j_range:
        if (row - j >= 0 and df.loc[row - j, column_name] == df.loc[row, column_name]) or (row + j < len(df) and df.loc[row + j, column_name] == df.loc[row, column_name]):
            if df.loc[row, 'failure_reason'] == '':
                df.loc[row, 'failure_reason'] = reason
            else:
                df.loc[row, 'failure_reason'] += f", {reason}"

# Loop to check and append failure reasons for 'hybrid', 'pano', and 'csa'
for i in range(len(df)):
    if df.loc[i, 'hybrid'] == 'hybrid':
        check_failure(df, 'hybrid', i, range(1, 2), 'hybrid')

    if df.loc[i, 'pano'] == 'pano':
        check_failure(df, 'pano', i, range(1, 4), 'pano')

    if df.loc[i, 'csa'] == 'csa':
        check_failure(df, 'csa', i, range(1, 5), 'csa')

# Fill remaining rows without failure reason with 'ok'
df['failure_reason'] = df['failure_reason'].replace('', 'ok')
# Recalculating 'status' based on the failure reason

df['status'] = df['failure_reason'].apply(lambda x: 'failed' if x != 'ok' else 'ok')


In [82]:
df

,id,hybrid,pano,csa,status,failure_reason
0,1,hybrid,pano,csa,failed,csa
1,2,non-hybrid,non-pano,non-csa,ok,ok
2,3,non-hybrid,non-pano,csa,failed,"csa, csa"
3,4,hybrid,non-pano,non-csa,ok,ok
4,5,non-hybrid,pano,non-csa,failed,"pano, pano"
5,6,non-hybrid,pano,non-csa,failed,pano
6,7,hybrid,pano,csa,failed,"pano, pano, csa"
